In [7]:
import import_ipynb
from ConsistencyIndexes import *

In [8]:
class ParamInvividual:
    def __init__(self,N=2,idx=[CRC,Border_Err],W=[1,1],M="mean"):
        self.NIdx = N
        self.Idx = idx
        self.Weights = W
        self.Mode = M
        self.Score = 0

In [9]:
@numba.jit
def CopyIndv(Inv):
    return ParamInvividual(Inv.NIdx,Inv.Idx,Inv.Weights,Inv.Mode)

In [10]:
@numba.jit
def Generator():
    Indexes = [Abs_Dif,Gray_Dif,MSE,Border_Err,CRC,Entropy_Dif,Freq_Dif,TSNR,TSSIM,OF,OF_B]
    Modes = ["mean","log","norm","Z"]
    Nindex = random.sample(range(0,len(Indexes)),random.randint(2,3))
    Idx = [Indexes[i] for i in Nindex]
    W = [random.uniform(0.1, 1) for _ in range(len(Nindex))]
    M = Modes[random.randint(0,len(Modes)-1)]
    return ParamInvividual(len(Nindex),Idx,W,M)

In [11]:
@numba.jit
def Gen_Population(N=100):
    return [Generator() for _ in range(N)]

In [12]:
@numba.jit
def Evaluation(Individual,Frames,Targets):
    Inc,Metric = InconsistentRegion(Frames,Mix_Metrics,Individual.Mode,Individual.Idx,Individual.Weights)
    Inc = Inc[1:]
    Dif = [abs(Inc[i]-Targets[i]) for i in range(len(Inc))]
    return np.mean(sum(Dif)/len(Dif)) #+ sum([0 if d.all()==np.zeros_like(Targets[0]).all() else 1 for d in Targets])# + Metric)

In [13]:
@numba.jit
def Evaluate_Population(Population,Frames,Targets):
    for i in range(len(Population)):
        print("Pop: ",i+1,'/',len(Population),end='\r')
        Population[i].Score = Evaluation(Population[i],Frames,Targets)

In [14]:
@numba.jit
def Read2Binary(path):
    T = read_images(path)
    T = [cv2.cvtColor(t,cv2.COLOR_RGB2GRAY) for t in T]
    for t in T:
        t[t>=150]=255
        t[t<150]=0
    return T

In [15]:
def Selection(Population,NTop=10):
    Scores = [P.Score for P in Population]
    Dictionary = dict(zip(Population, Scores))
    Sorted = sorted(Dictionary.items(), key=lambda x: x[1])
    return [key for key, value in Sorted[:NTop]]

In [16]:
@numba.jit
def Cross(Indv1,Indv2):
    N = (Indv1.NIdx+Indv2.NIdx)//2
    Dic = dict(zip(Indv1.Idx+Indv2.Idx,Indv1.Weights+Indv2.Weights))
    idx = random.sample(list(Dic.keys()), N)
    W = [Dic[key] for key in idx]
    M = [Indv1.Mode,Indv2.Mode][random.randint(0,1)]
    return ParamInvividual(N,idx,W,M)

In [17]:
@numba.jit
def Mutation(Indv):
    I = CopyIndv(Indv)
    if random.randint(0,1)==0:
        I.Weights = [random.uniform(0.1, 10) for _ in range(Indv.NIdx)]
        return I
    else:
        return Generator()

In [18]:
@numba.jit
def Gen_From_Top(N,Top):
    Population = []
    for n in range(N):
        Op = random.randint(0,1)
        if Op==0:
            I = Cross(Top[random.randint(0,len(Top)-1)],Top[random.randint(0,len(Top)-1)])
        else:
            I = Mutation(Top[random.randint(0,len(Top)-1)])
        Population.append(I)
    return Population

In [19]:
@numba.jit
def Train(Frames,Targets,NPop=5,Generations=3,NTop=3):
    Population = Gen_Population(NPop)
    Evaluate_Population(Population,Frames,Targets)
    for G in range(Generations):
        print("Gen: ",G+1,"/",Generations,'\n')
        Top = Selection(Population,NTop)
        Children = Gen_From_Top(NPop-NTop,Top)
        Evaluate_Population(Children,Frames,Targets)
        Population = Top+Children
    Top = Selection(Population,NTop)
    return Top[0]

In [20]:
# If Mask is zeros add to diff

In [21]:
Frames = read_images("saved_frames")

In [22]:
I,B = AddInc(Frames)

In [23]:
len(I)

69

In [24]:
Best = Train(I[:15],B[:14],50,25,20)

Gen:  1 / 25 

Gen:  2 / 25 

Gen:  3 / 25 

Gen:  4 / 25 

Gen:  5 / 25 

Gen:  6 / 25 

Gen:  7 / 25 

Gen:  8 / 25 

Gen:  9 / 25 

Gen:  10 / 25 

Gen:  11 / 25 

Gen:  12 / 25 

Gen:  13 / 25 

Gen:  14 / 25 

Gen:  15 / 25 

Gen:  16 / 25 

Gen:  17 / 25 

Gen:  18 / 25 

Gen:  19 / 25 

Gen:  20 / 25 

Gen:  21 / 25 

Gen:  22 / 25 

Gen:  23 / 25 

Gen:  24 / 25 

Gen:  25 / 25 



In [25]:
Best.Mode,Best.Idx,Best.Weights

('mean',
 [CPUDispatcher(<function Border_Err at 0x000002058DA6DB88>),
  CPUDispatcher(<function TSSIM at 0x000002058DA6D4C8>)],
 [0.5311124470843562, 0.9750046438951553])

In [27]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,Best.Mode,Best.Idx,Best.Weights)
ThroughFrames(Img)

KeyboardInterrupt: 

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,"mean",[CRC,Gray_Dif],[0.6206048025607019, 0.508216164936656])
ThroughFrames(Img)

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,"log",[MSE,Entropy_Dif,CRC], [0.6795718573404556, 0.3460195810616382, 0.9312673477407468])
ThroughFrames(Img)

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,"log",[Entropy_Dif,CRC],  [0.7347384902639871, 0.3513275883482687])
ThroughFrames(Img)

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,"log",[CRC,Abs_Dif,Gray_Dif],  [0.3992279845080944, 0.9296664253181617, 0.7183341482172553])
ThroughFrames(Img)

('mean',
 [<function ConsistencyIndexes.CRC(image1, image2)>,
  <function ConsistencyIndexes.Gray_Dif(image1, image2)>],
 [0.6206048025607019, 0.508216164936656])

('log',
 [<function ConsistencyIndexes.MSE(image1, image2)>,
  <function ConsistencyIndexes.Entopy_Dif(image1, image2)>,
  <function ConsistencyIndexes.CRC(image1, image2)>],
 [0.6795718573404556, 0.3460195810616382, 0.9312673477407468])

('mean',
 [<function ConsistencyIndexes.Gray_Dif(image1, image2)>,
  <function ConsistencyIndexes.CRC(image1, image2)>],
 [0.22467344683611584, 0.5673037121649521])

('log',
 [<function ConsistencyIndexes.Entopy_Dif(image1, image2)>,
  <function ConsistencyIndexes.CRC(image1, image2)>],
 [0.7347384902639871, 0.3513275883482687])

('log',
 [CPUDispatcher(<function CRC at 0x0000023531086CA8>),
  CPUDispatcher(<function Abs_Dif at 0x0000023531065B88>),
  CPUDispatcher(<function Gray_Dif at 0x000002353106AEE8>)],
 [0.3992279845080944, 0.9296664253181617, 0.7183341482172553])

('mean',
 [CPUDispatcher(<function Border_Err at 0x000002058DA6DB88>),
  CPUDispatcher(<function TSSIM at 0x000002058DA6D4C8>)],
 [0.5311124470843562, 0.9750046438951553])

Seguimiento de error en frames consecutivos<br>
Filtros diferentes de cartoonization<br>
OF en bordes<br>
Fracuencias en Imagenes<br>
Diferencia en imagen real y caricaturizada<br>
